##Libraries and environment preparation

In [1]:
#Colab Environment Check for GPU and RAM
!nvidia-smi

Thu Mar 31 01:33:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#Install essential packages
%%capture
!pip install datasets transformers rouge-score nltk wandb
!apt install git-lfs

In [3]:
# Make sure your version of Transformers is at least 4.11.0 
# to run the following code correctly:
import datasets
import transformers
print(transformers.__version__)

4.17.0


In [4]:
from transformers import AutoTokenizer    
# Huggingface Automodel class
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "t5-small"

In [5]:
# Import Wandb 
import os
import wandb
API_KEY = '39991c538626bee25c64d4f8a4c3403dd635537c'
os.environ["WANDB_API_KEY"] = API_KEY

##Load the dataset

In [6]:
# import dataset
raw_datasets = datasets.load_dataset("xsum")

Using custom data configuration default


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

##Preprocess the data

In [8]:
# Import tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [9]:
# tokenlize inputs into map
prefix = "summarize: "
max_input_length = 512
max_target_length = 64

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_train = raw_datasets["train"].map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["document", "summary", "id"])

  0%|          | 0/205 [00:00<?, ?ba/s]

In [11]:
tokenized_val = raw_datasets["validation"].map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["document", "summary", "id"])

  0%|          | 0/12 [00:00<?, ?ba/s]

In [12]:
tokenized_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 204045
})

##Fine-tuning the model

In [13]:
# Import tokenizer from model checkpoint and print detail
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [14]:
# Define traing args, batch size and epoch
# batch size max 8 for input length 1024 on Colab Pro

batch_size = 8
epochs = 1
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy = "epoch",
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    #report_to="wandb",
)

In [15]:
import nltk
import numpy as np
nltk.download('punkt')

metric = datasets.load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
# Pass into the trainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [17]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 204045
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 25506
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: shusunny (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.631500,2.400149,29.489200,8.535400,23.329400,23.331000,18.815800


***** Running Evaluation *****
  Num examples = 11332
  Batch size = 8
Saving model checkpoint to t5-small-finetuned-xsum/checkpoint-25506
Configuration saved in t5-small-finetuned-xsum/checkpoint-25506/config.json
Model weights saved in t5-small-finetuned-xsum/checkpoint-25506/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-xsum/checkpoint-25506/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-xsum/checkpoint-25506/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=25506, training_loss=2.693736815596528, metrics={'train_runtime': 5522.1593, 'train_samples_per_second': 36.95, 'train_steps_per_second': 4.619, 'total_flos': 2.754899075216179e+16, 'train_loss': 2.693736815596528, 'epoch': 1.0})

In [18]:
wandb.finish()

eval/gen_len,▁
eval/loss,▁
eval/rouge1,▁
eval/rouge2,▁
eval/rougeL,▁
eval/rougeLsum,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████


In [19]:
!ls -lh t5-small-finetuned-xsum/checkpoint-25506

total 695M
-rw-r--r-- 1 root root 1.4K Mar 31 03:13 config.json
-rw-r--r-- 1 root root 462M Mar 31 03:13 optimizer.pt
-rw-r--r-- 1 root root 231M Mar 31 03:13 pytorch_model.bin
-rw-r--r-- 1 root root  15K Mar 31 03:13 rng_state.pth
-rw-r--r-- 1 root root  559 Mar 31 03:13 scaler.pt
-rw-r--r-- 1 root root  623 Mar 31 03:13 scheduler.pt
-rw-r--r-- 1 root root 1.8K Mar 31 03:13 special_tokens_map.json
-rw-r--r-- 1 root root 1.9K Mar 31 03:13 tokenizer_config.json
-rw-r--r-- 1 root root 2.4M Mar 31 03:13 tokenizer.json
-rw-r--r-- 1 root root 6.8K Mar 31 03:13 trainer_state.json
-rw-r--r-- 1 root root 3.2K Mar 31 03:13 training_args.bin


In [20]:
!zip -r t5-small-finetuned-xsum.zip /content/t5-small-finetuned-xsum/checkpoint-25506/

  adding: content/t5-small-finetuned-xsum/checkpoint-25506/ (stored 0%)
  adding: content/t5-small-finetuned-xsum/checkpoint-25506/scaler.pt (deflated 55%)
  adding: content/t5-small-finetuned-xsum/checkpoint-25506/scheduler.pt (deflated 49%)
  adding: content/t5-small-finetuned-xsum/checkpoint-25506/tokenizer_config.json (deflated 80%)
  adding: content/t5-small-finetuned-xsum/checkpoint-25506/optimizer.pt (deflated 7%)
  adding: content/t5-small-finetuned-xsum/checkpoint-25506/config.json (deflated 62%)
  adding: content/t5-small-finetuned-xsum/checkpoint-25506/pytorch_model.bin (deflated 8%)
  adding: content/t5-small-finetuned-xsum/checkpoint-25506/tokenizer.json (deflated 74%)
  adding: content/t5-small-finetuned-xsum/checkpoint-25506/special_tokens_map.json (deflated 83%)
  adding: content/t5-small-finetuned-xsum/checkpoint-25506/rng_state.pth (deflated 27%)
  adding: content/t5-small-finetuned-xsum/checkpoint-25506/trainer_state.json (deflated 79%)
  adding: content/t5-small-fin

In [21]:
!cp t5-small-finetuned-xsum.zip '/content/drive/My Drive/weights/'